In [15]:
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import statistics
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from functools import partial
import warnings

warnings.filterwarnings('ignore')


In [16]:
sys.path.append(os.path.abspath("/home/bk_anupam/code/ML/ML_UTILS/"))

In [17]:
import train_tabular_utils as tt
import cv_split_utils
import enums
import data_utils

In [18]:
RUN_MODE = "LOCAL"
DATA_READPATH = "./data/"
DATA_WRITEPATH = "./data/"
if RUN_MODE == "KAGGLE":    
    DATA_READPATH = "/kaggle/input/playground-series-s4e6/"
    DATA_WRITEPATH = "/kaggle/working/"

In [19]:
# import train dataset locally from data folder
df_train = pd.read_csv(DATA_READPATH + "train.csv")
# import test dataset locally from data folder
df_test = pd.read_csv(DATA_READPATH + "test.csv")

In [20]:
def extract_engine_info(engine_desc: str, brand: str) -> dict:
    # check if engine_desc is a valid string
    if not isinstance(engine_desc, str):
        return None
    engine_desc = engine_desc.lower()
    brand = brand.lower()
    # Define patterns for each attribute
    horsepower_pattern = r'(\d+(\.\d+)?\s*)hp'
    capacity_pattern = r'(\d+(\.\d+)?\s*)l'
    cylinders_pattern = r'(\d+)\s*cylinder|v(\d+)'
    fuel_pattern = r'(gasoline|diesel|flex|electric|dohc|ohv)'    
    # Extract horsepower
    horsepower_match = re.search(horsepower_pattern, engine_desc)
    horsepower = float(horsepower_match.group(1)) if horsepower_match else None
    # Extract capacity
    capacity_match = re.search(capacity_pattern, engine_desc)
    capacity = float(capacity_match.group(1)) if capacity_match else None
    # Extract cylinders
    cylinders_match = re.search(cylinders_pattern, engine_desc)    
    cylinders = int(cylinders_match.group(1) or cylinders_match.group(2)) if cylinders_match else None
    # Extract fuel type
    fuel_match = re.search(fuel_pattern, engine_desc)
    fuel = fuel_match.group(0) if fuel_match else None
    if fuel in ('dohc', 'ohv'):
        fuel = "gasoline"
    # check is fuel_type is None and brand is "Tesla" then set fuel_type to "electric"
    if fuel is None and brand == "tesla":
        fuel = "electric"
    # Extract turbo
    turbo_match = re.search(r'turbo', engine_desc)
    turbo = True if turbo_match else False
    # extract hybrid
    hybrid_match = re.search(r'hybrid', engine_desc)
    hybrid = True if hybrid_match else False
    return {
        "horsepower": horsepower, 
        "capacity": capacity,
        "cylinders": cylinders,
        "fuel": fuel,
        "turbo": turbo,
        "hybrid": hybrid
    }

In [21]:
def extract_transmission_info(transmission_desc: str) -> dict:
    if not isinstance(transmission_desc, str):
        return None
    transmission_desc = transmission_desc.lower()
    patterns = {
        "transmission_speed": r"(\d+)[\s-]speed",
        "auto": r"automatic|cvt|a/t|\sat|transmission overdrive switch",
        "manual": r"manual|m/t|\smt",
        "single_speed": r"single-speed"
    }
    transmission_info = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, transmission_desc)
        if key == "transmission_speed":
            transmission_info[key] = int(match.group(1)) if match else None
        elif key == "auto" and match:
            transmission_info["transmission_type"] = "automatic"
            break
        elif key == "manual" and match:
            transmission_info["transmission_type"] = "manual"
            break
        elif key == "single_speed" and match:
            transmission_info["transmission_type"] = "single_speed"
            break

    if "transmission_type" not in transmission_info:
        transmission_info["transmission_type"] = "automatic" if transmission_desc == "f" else "Unknown"

    return transmission_info

In [22]:
# apply the "extract_engine_info" function to the 'engine' and 'brand' columns and extract each of the returned values into new columns
df_train[['horsepower', 'capacity', 'cylinders', 'fuel', 'turbo', 'hybrid']] = \
    df_train.apply(lambda x: extract_engine_info(x['engine'], x['brand']), axis=1).apply(pd.Series)
df_test[['horsepower', 'capacity', 'cylinders', 'fuel', 'turbo', 'hybrid']] = \
    df_test.apply(lambda x: extract_engine_info(x['engine'], x['brand']), axis=1).apply(pd.Series)
# apply the "extract_transmission_info" function to the 'transmission' column and extract each of the returned values into new columns
df_train[['transmission_speed', 'transmission_type']] = \
    df_train['transmission'].apply(extract_transmission_info).apply(pd.Series)
df_test[['transmission_speed', 'transmission_type']] = \
    df_test['transmission'].apply(extract_transmission_info).apply(pd.Series)
# by subtract model_year from current year create a new feature "age"
df_train['age'] = 2024 - df_train['model_year']
df_test['age'] = 2024 - df_test['model_year']

# count missing values in each new column
na_counts = df_train[['cylinders', 'fuel', 'horsepower', 'capacity', 'transmission_speed', 'transmission_type', 'age']].isna().sum()
for col, count in na_counts.items():
    print(f'{col}_na_count = {count}')

cylinders_na_count = 21763
fuel_na_count = 8689
horsepower_na_count = 33259
capacity_na_count = 6698
transmission_speed_na_count = 84377
transmission_type_na_count = 0
age_na_count = 0


#### Handle Missing Values

In [23]:
def extract_fuel_type(engine_desc: str) -> str:    
    fuel_type_pattern = r'(Gasoline|Diesel|Flex|Electric|DOHC|OHV|SOHC|Dual Motor)'
    # Extract fuel type
    fuel_type_match = re.search(fuel_type_pattern, engine_desc)
    fuel_type = fuel_type_match.group(0) if fuel_type_match else None
    if fuel_type in ('DOHC', 'OHV', 'SOHC'):
        fuel_type = "Gasoline"
    elif fuel_type == 'Dual Motor':
        fuel_type = "Electric"
    return fuel_type

In [24]:
df_train_fueltype_na = df_train[df_train.fuel_type.isna()]
df_test_fueltype_na = df_test[df_test.fuel_type.isna()]
df_train_fueltype_na.loc[:, "fuel_type"] = df_train_fueltype_na["engine"].apply(extract_fuel_type)
df_test_fueltype_na.loc[:, "fuel_type"] = df_test_fueltype_na["engine"].apply(extract_fuel_type)
# update missing fuel_type in df_train from df_fueltype_na by joining on id
df_train = df_train.merge(df_train_fueltype_na[["id", "fuel_type"]], on="id", how="left")
# drop fuel_type column from df_train_fueltype_na
df_train.drop("fuel_type_y", axis=1, inplace=True)
df_train.rename(columns={"fuel_type_x": "fuel_type"}, inplace=True)
df_test = df_test.merge(df_test_fueltype_na[["id", "fuel_type"]], on="id", how="left")
df_test.drop("fuel_type_y", axis=1, inplace=True)
df_test.rename(columns={"fuel_type_x": "fuel_type"}, inplace=True)

In [25]:
# impute "clean_title", "accident", "fuel_type" columns missing values with "Unknown"
df_train.fillna({"clean_title": "Unknown", "accident": "Unknown", "fuel_type": "Unknown", "fuel": "Unknown"}, inplace=True)
df_test.fillna({"clean_title": "Unknown", "accident": "Unknown", "fuel_type": "Unknown", "fuel": "Unknown"}, inplace=True)

In [26]:
df_train.to_csv(DATA_WRITEPATH + "train_preprocessed.csv", index=False)
df_test.to_csv(DATA_WRITEPATH + "test_preprocessed.csv", index=False)
df_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,price,horsepower,capacity,cylinders,fuel,turbo,hybrid,transmission_speed,transmission_type,age
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,...,4200,172.0,1.6,4.0,gasoline,False,False,NaN,automatic,17
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,...,4999,252.0,3.9,8.0,gasoline,False,False,NaN,automatic,22
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,...,13900,320.0,5.3,8.0,flex,False,False,NaN,automatic,22
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,...,45000,420.0,5.0,8.0,gasoline,False,False,NaN,Unknown,7
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,...,97500,208.0,2.0,4.0,gasoline,False,False,7.0,automatic,3
